In [9]:
import json
import numpy as np
import os

In [10]:
texts = []
labels = []
#실행 파일과 같은 폴더에 파일들이 존재해야 합니다.
for i in range(1, 10):
    file_name = f'기술_과학_{i}.json'  
    
    if os.path.exists(file_name):  # 파일이 존재하는 경우에만 처리
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        for conversation in data["dataset"].get("conversations", []):
            for utterance in conversation.get("utterances", []):
                text = utterance.get("utterance_text", "")
                evaluation = utterance.get("utterance_evaluation", [])
                if evaluation:  # 평가 대상이 아닌 문장을 제외
                    texts.append(text)
                    labels.append(evaluation)

In [11]:
#토큰화 코드: 라이브러리 사용
import konlpy 
from konlpy.tag import Okt, Hannanum, Kkma 

okt = Okt()
tokenized_texts = []

for i in range(len(texts)):
    tokenized_texts.append(okt.morphs(texts[i]))

In [12]:
#word2vec 과정
_word_to_id = {}
_id_to_word = {}

def preprocess(words): #앞에서 토큰화를 해서 적절히 바꿨습니다. 
    for word in words:
        if word not in _word_to_id:
            new_id = len(_word_to_id)
            _word_to_id[word] = new_id
            _id_to_word[new_id] = word
    corpus = np.array([_word_to_id[w] for w in words])
    return corpus, _word_to_id, _id_to_word

In [13]:
attribute_labels = {
    'linguistic_acceptability': [],
    'consistency': [],
    'interestingness': [],
    'unbias': [],
    'harmlessness': [],
    'no_hallucination': [],
    'understandability': [],
    'sensibleness': [],
    'specificity': []
}

# 속성별 레이블 추출
for i in range(len(labels)):
    # 현재 대화에 대한 속성값 초기화
    attribute_values = {key: 0 for key in attribute_labels.keys()}

    for j in range(len(labels[i])):
        for key in attribute_labels.keys():
            if labels[i][j].get(key) == 'yes':
                attribute_values[key] += 1

    # 각 속성별로 모든 발화에서 'yes'인 경우만 1, 아니면 0
    for key in attribute_labels.keys():
        if attribute_values[key] == len(labels[i]):
            attribute_labels[key].append(1)
        else:
            attribute_labels[key].append(0)

In [21]:
# 코드 참고: 강의 코드. 
#https://github.com/ssuai/deep_learning_from_scratch2/tree/master/ch06

#rnnlm
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
from dataset import ptb
from rnnlm import Rnnlm, Rnnlmv2, BetterRnnlm

In [15]:
batch_size = 20
wordvec_size = 100
hidden_size = 50  # RNN의 은닉 상태 벡터의 원소 수
time_size = 5     # RNN을 펼치는 크기
lr = 20.0
max_epoch = 2
max_grad = 0.25

In [16]:
#RnnLM에서 계층 생성 부분에 마지막에 sigmoid 함수를 추가
#rnnlm파일 열어서 아마 TimeSigmoidWithLoss 추가일듯. 

model = Rnnlm(wordvec_size = wordvec_size, hidden_size = hidden_size )
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)
results = []

for i in range(len(tokenized_texts)): #마지막은 테스트로 사용
    corpus, word_to_id, id_to_word = preprocess(tokenized_texts[i])
    xs = corpus[:-1]
    ts = corpus[1:]
    trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,eval_interval=20)
    results.append(eval_perplexity(model, corpus) / len(corpus))

퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼플렉서티 평가 중 ...
999 / 10001
퍼

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

results = np.array(results).reshape(-1, 1)

for key in attribute_labels.keys():
    try: 
        X_train, X_test, y_train, y_test = train_test_split(results, attribute_labels[key], test_size=0.3, random_state=42)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        y_pred  = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(key, accuracy)
    except:
        print(key, '에서 에러가 발생') #값이 모두 yes인 경우

linguistic_acceptability 0.8636363636363636
consistency 에서 에러가 발생
interestingness 1.0
unbias 에서 에러가 발생
harmlessness 에서 에러가 발생
no_hallucination 0.9545454545454546
understandability 1.0
sensibleness 0.9090909090909091
specificity 1.0


In [20]:
model = Rnnlmv2(wordvec_size = wordvec_size, hidden_size = hidden_size )
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)
results = []

for i in range(len(tokenized_texts)): #마지막은 테스트로 사용
    corpus, word_to_id, id_to_word = preprocess(tokenized_texts[i])
    xs = corpus[:-1]
    ts = corpus[1:]
    trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,eval_interval=20)
    results.append(eval_perplexity(model, corpus) / len(corpus))


results = np.array(results).reshape(-1, 1)

for key in attribute_labels.keys():
    try: 
        X_train, X_test, y_train, y_test = train_test_split(results, attribute_labels[key], test_size=0.3, random_state=42)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        y_pred  = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(key, accuracy)
    except:
        print(key, '에서 에러가 발생') #값이 모두 yes인 경우

퍼플렉서티 평가 중 ...


TypeError: TimeSigmoidWithLoss.forward() missing 1 required positional argument: 'ts'

In [22]:
model = BetterRnnlm(wordvec_size = wordvec_size, hidden_size = hidden_size )
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)
results = []

for i in range(len(tokenized_texts)): #마지막은 테스트로 사용
    corpus, word_to_id, id_to_word = preprocess(tokenized_texts[i])
    xs = corpus[:-1]
    ts = corpus[1:]
    trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,eval_interval=20)
    results.append(eval_perplexity(model, corpus) / len(corpus))


results = np.array(results).reshape(-1, 1)

for key in attribute_labels.keys():
    try: 
        X_train, X_test, y_train, y_test = train_test_split(results, attribute_labels[key], test_size=0.3, random_state=42)
        model = LogisticRegression()
        model.fit(X_train, y_train)
        y_pred  = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(key, accuracy)
    except:
        print(key, '에서 에러가 발생') #값이 모두 yes인 경우

퍼플렉서티 평가 중 ...


ValueError: shapes (350,50) and (100,10000) not aligned: 50 (dim 1) != 100 (dim 0)